# Download Active Listings
Etsy API limits the offset to 12,000 listings, so that is the maximum number of recently updated listings that can be accessed

In [1]:
import requests
import pandas as pd
import json

Get all listings that match keywords "stoneware" and "handmade"

In [2]:
#GetListings URL and Parameters for the API
url = "https://openapi.etsy.com/v3/application/listings/active"
key = "r8ymjsnfohuidr1wev7e4cfg"
keyword = "stoneware,handmade"

In [3]:
#initialize the dataframes
dataframe = pd.DataFrame()
df = pd.DataFrame()

### 3/24 I changed this loop a little and need to re-run it

In [4]:
#loop that increases the offset by 100 each time due to the API limitation of downloading a maximum of 100 listings at a time
limit = 100
offset = 0
for i in range (1,121):
    parameters = {"client_id":key, "limit":limit, "offset":offset, "keywords":keyword}
    download = requests.get(url, params=parameters)
    data = download.json()
    if "error" in data.keys():
        print("Maximum Offset Reached")
    else:
        dataframe = pd.json_normalize(data['results'])
        df = df.append(dataframe, ignore_index = True)
    offset = offset + limit
    i = i+1
    

Maximum Offset Reached
Maximum Offset Reached
Maximum Offset Reached
Maximum Offset Reached


In [11]:
#confirm that there are 12,000 rows
len(df)

500

In [6]:
#write the data to a csv file
#df.to_csv('12000listingsStonewareHandmade032222.csv')

## Download Shop Info for all the Etsy listings in the data



In [8]:
#NOTE: You dont need this if you are doing downloading the 12,000 listings and the shop info at the same time
#df = pd.read_csv('12000listingsStonewareHandmade032222.csv')

In [20]:
#Get all the unique shop IDs
shop_ids = df["shop_id"].tolist()
shop_list = list(set(shop_ids))

In [23]:
#initialize the dataframes
dataframeShop = pd.DataFrame()
dfShop = pd.DataFrame()

In [37]:
#loop that queries the API for 
limit = 100

for j in range(0, len(shop_list)):
    shop_id = str(shop_list[j])
    url = "https://openapi.etsy.com/v3/application/shops/" + shop_id
    parameters = {"client_id":key, "limit":limit}
    download = requests.get(url, params=parameters)
    dataShops = download.json()
    dataframeShop = pd.json_normalize(dataShops)
    dfShop = df.append(dataframeShop, ignore_index = True)

In [38]:
dataframeShop

,shop_id,shop_name,user_id,create_date,title,announcement,currency_code,is_vacation,vacation_message,sale_message,...,policy_shipping,policy_refunds,policy_additional,policy_seller_info,policy_update_date,policy_has_private_receipt_info,has_unstructured_policies,policy_privacy,review_average,review_count
0,9606653,MuddyMountainPottery,49320080,1402345722,Handmade Stoneware Pottery from Wyoming,"Muddy Mountain Pottery offers unique, function...",USD,False,None,Thanks so much for your order! We&#39;ll emai...,...,Most of our pottery is made to order and requi...,In the (unlikely!) event that something is los...,We welcome wholesale orders. Please contact u...,None,1427385480,False,True,None,5,8


## Merge the data

In [27]:
#create a dataframe with just the columns I'm interested in for the listings data
listings = df[['listing_id', 'shop_id', 'title',
       'description', 'num_favorers',
       'listing_type', 'tags', 'materials',
       'style','taxonomy_id','price.amount',
       'price.divisor', 'price.currency_code','creation_timestamp', 'ending_timestamp',
       'original_creation_timestamp','quantity']]

In [39]:
#create a dataframe with just the columns I'm interested in for the shops data
shops = dfShop[['shop_id', 'shop_name', 'create_date', 'title',
       'announcement', 'currency_code', 'is_vacation', 'vacation_message',
       'sale_message',
       'listing_active_count',
       'accepts_custom_requests', 'url',
       'num_favorers', 'languages', 'is_shop_us_based', 'transaction_sold_count',
       'shipping_from_country_iso', 'shop_location_country_iso',
       'review_average', 'review_count']]

In [40]:
#merge the two datasets so that the shop information is added to each listing
data = pd.merge(listings, shops, on="shop_id")

In [ ]:
# save the merged data
#data.to_csv('mergedData.csv')